In [30]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import nltk
from nltk.tokenize import word_tokenize
from datasets import load_dataset

In [74]:
### Hyper Params
batch_size = 1
hidden_size = 120
input_size = 1
lr = 0.001
epochs = 1

In [3]:
raw_dataset = load_dataset("jaydenccc/AI_Storyteller_Dataset", split="train")['short_story']

README.md:   0%|          | 0.00/404 [00:00<?, ?B/s]

(…)-00000-of-00001-532ad934f217d092.parquet:   0%|          | 0.00/130k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [4]:
nltk.download('punkt_tab')

text_corpus = ""
for item in raw_dataset:
    text_corpus += item.lower()

tokens = word_tokenize(text_corpus)
unique_words = set(tokens)
word_to_num = {word: idx for idx, word in enumerate(unique_words)}
num_to_word = {idx: word for idx, word in enumerate(unique_words)}

[nltk_data] Downloading package punkt_tab to /home/grant/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [20]:
output_size = len(word_to_num)

pad_tok_id = output_size
num_to_word[pad_tok_id] = '<pad>'
word_to_num['<pad>'] = pad_tok_id

In [7]:
class StoryDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [28]:
pads = ['<pad>' for i in range(9)]
corpus = text_corpus.split(' ')

train_dataset = StoryDataset(corpus[:25000])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

test_dataset = StoryDataset(corpus[25000:])
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [29]:
for i, batch in enumerate(train_dataloader):
    # Give model i word, output should be id of next word. So loss will be item+1
    if i > 10:
        break
    print(batch)

['detective', 'emily', 'had', 'always', 'dreamt', 'of', 'getting', 'an', 'exciting', 'case']
['to', 'solve.', 'her', 'wish', 'came', 'true', 'when', 'she', 'received', 'a']
['call', 'about', 'a', 'series', 'of', 'bizarre', 'murders', 'in', 'a', 'small']
['town', 'of', 'willow', 'creek.', 'emily', 'had', 'heard', 'about', 'this', 'town,']
['known', 'for', 'its', 'friendly', 'community', 'and', 'green', 'landscape.', 'she', 'had']
['never', 'imagined', 'such', 'horror', 'could', 'lurk', 'in', 'these', 'picturesque', 'streets.\n\nemily']
['arrived', 'at', 'the', 'crime', 'scene,', 'a', 'small', 'cottage', 'on', 'the']
['outskirts', 'of', 'town.', 'from', 'the', 'outside,', 'it', 'looked', 'like', 'any']
['other', 'quaint', 'home,', 'but', 'inside,', 'it', 'was', 'a', 'bloodbath.', 'three']
['people', 'had', 'been', 'brutally', 'murdered,', 'and', 'the', 'sight', 'was', 'enough']
['to', 'make', 'her', 'stomach', 'turn.', 'there', 'were', 'no', 'signs', 'of']


In [73]:
class RNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Init hidden state (batch size * hidden size)
        batch_size = x.size(0)
        hidden = torch.zeros(1, batch_size, self.hidden_size) # 

        # Forward through net
        out, hidden = self.rnn(x, hidden)

        # Only use last hidden state for output
        out = self.fc(out[:, -1, :])
        return out

In [77]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 120),
            nn.ReLU(),
            nn.Linear(120, 240),
            nn.ReLU(),
            nn.Linear(240, output_size)
        )
    def forward(self, x):
        x = self.fc(x)
        return torch.argmax(x, dim=0)

In [78]:
net = Net()
crit = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(), lr=lr)

In [79]:
out = net(torch.tensor([word_to_num['upon']], dtype=torch.float32))

In [82]:
net.train()
for epoch in range(epochs):
    for i, item in enumerate(train_dataset):
        # opt.zero_grad()
        # output = net(torch.tensor([word_to_num[item]], dtype=torch.float32))
        x = torch.tensor([word_to_num[train_dataset[i+1]]], dtype=torch.float32)
        # loss = crit(output, torch.tensor([word_to_num[train_dataset[i+1]]], dtype=torch.float32))
        # loss.backward()
        # opt.step()
        # print(loss.item())

KeyError: 'solve.'

In [84]:
word_to_num['solve']

1124